首先恭喜各位，通过第一轮的简历筛选，进入线上测试环节。

通过这次线上测试，希望大家能够在限定的时间内，尽自己最大努力，完成一些数据处理和机器学习建模任务，展现自己对机器学习建模工作的熟悉程度。

## 任务说明
请在这个jupyter notebook中
* 补充完成任务所需要的python代码。可以直接在code cell中编写代码；或者将代码写在.py文件中后在这个notebook中import使用
* 利用markdown cell回答相关的问题或者解释分析思路

完成后，在自己的github上建立一个repo。将这个notebook，相关的.py文件，保存处理结果的csv文件等，push到建立的repo。将repo链接提交给面试官。


## 注意事项：
1. 使用python3.6以上版本，如果使用```pandas```等python库，请先升级到最新版本。
2. 请确保面试官在clone各位发布的repo后，能够跑通这个notebook中的代码。
3. 除了实现任务所需的功能，编写的代码具有高可维护性，也是我们非常希望候选人具备的能力。请使用可读性强的命名，避免代码冗余，以及体现其他良好的编程实践。（可参考Refactoring by Kent Beck and Martin Fowler一书中关于"bad code smell"的讨论）
4. 这个测试的目的是检验候选人是否具备高效的完成建模相关工作的能力，对相关工具的熟悉程度也是考核的目标之一。不鼓励重复造轮子。建议充分利用```pandas```, ```sklearn```等已有python库完成任务。
5. 请展现自己真实的能力，不要请人代劳。如果候选人试用期阶段表现的实际工作能力明显低于线上测试表现的能力，候选人会被认为采用非诚信手段通过线上测试，将予以坚决辞退。
6. 职场中，Deadline是非常严肃的。无论完成度如何，请务必在截止时间前，提交自己的结果。逾期提交将不予受理。

## 机器学习建模（任务1～任务2）
以下任务基于虚拟的建模数据dataset.csv

建模数据集构成
* id：样本的编号
* dataset：划分了哪些数据用于train, 那些用于test
* label:  0或者1, 只有train数据才给出了label
* 其实字母f开头的列为建模特征

In [ ]:
import pandas as pd
model_data = pd.read_csv('model_data.csv')

### 任务1

请根据train数据，使用3种指定的机器学习算法分别训练模型。这3种算法是
* xgboost
* logistic regression
* random forest

要求:
* 预测目标为label列
* 模型优化的目标为auc_roc
* 需要对特征进行筛选。每种算法训练的模型，最后使用的特征控制在20个以内。
* 每种算法得到的模型，将模型在test数据集的预测结果(0~1之间的一个分值)保存在prediciton.csv文件中。在候选人提交结果后，面试官会计算候选人的模型在test数据集的auc_roc。 csv应该包含id, 以及每个模型对应的proba。请参考作为样例的prediciton_sample.csv中的形式

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [16]:
model_data = pd.read_csv('model_data.csv')

def data_process(data, select_feature=np.arange(0,2723),fillna=False):
    """
    将原始数据的特征进行筛选并将其转化为train,val,test三个数据集
    :param data: dataframe 原始数据
    :param select_feature 数组 选择好的特征 
    :param fillna: 是否对缺失值进行填充
    :return: X_train, X_val, X_test, y_train, y_val
    """
    train_data = data[data['dataset'] == 'train']
    test_data = data[data['dataset'] == 'test']
    label = train_data['label']
    feature = train_data.iloc[:, select_feature]
    X_test = test_data.iloc[:, select_feature]
    if fillna:
        feature = feature.fillna(-1)
        X_test = X_test.fillna(-1)
    X_train, X_val, y_train, y_val = train_test_split(feature, label, test_size = 0.3, random_state = 666)
    return X_train, X_val, X_test, y_train, y_val

X_train, X_val, X_test, y_train, y_val = data_process(model_data)

In [3]:
# Xgboost可以对缺失值进行处理，构建一个Xgboost模型，并将其作为特征的筛选工具
model = XGBClassifier(learning_rate=0.03,
                      n_estimators=32,
                      max_depth=6,
                      gamma=0.5,
                      min_child_weight=3,
                      eta=0.3,
                      alpha=0.5,
                      subsample=0.3,
                      colsample_btree=0.5,
                      random_state=66
                      )
# 对所有特征进行训练
model.fit(X_train, y_train)

# 对train和test集进行预测，并评估AUC
y_pred = model.predict(X_val)
print("Accuracy : %.4g" % metrics.accuracy_score(y_val, y_pred))
y_train_proba = model.predict_proba(X_train)[:,1]
print("AUC Score (Train): %f" % metrics.roc_auc_score(y_train, y_train_proba))
y_proba = model.predict_proba(X_val)[:,1]
print("AUC Score (Test): %f" % metrics.roc_auc_score(y_val, y_proba))


Accuracy : 0.9289
AUC Score (Train): 0.941811
AUC Score (Test): 0.900249


In [20]:
# 根据上述Xgboost模型选择出特征重要度前20的特征
thresholds = sorted(model.feature_importances_)
results=pd.DataFrame()
results['columns'] = X_train.columns
results['importance'] = model.feature_importances_
results.sort_values(by='importance',ascending=False,inplace=True)
selected_features = results[:20]['columns'].index
# 筛选后的特征用于后续的训练
X_train_select, X_val_select, X_test_select, y_train_select, y_val_select = data_process(
    model_data,selected_features,fillna=True)


In [25]:
# Xgboost模型
model_xgboost = XGBClassifier(learning_rate=0.03,
                      n_estimators=64,
                      max_depth=6,
                      gamma=0.1,
                      min_child_weight=3,
                      eta=0.3,
                      alpha=0.1,
                      subsample=0.7,
                      colsample_btree=0.7,
                      random_state=66
                      )

model_xgboost.fit(X_train_select, y_train_select)
# 预测
y_pred_select_xgboost = model_xgboost.predict(X_val_select)
print("Accuracy : %.4g" % metrics.accuracy_score(y_val_select, y_pred_select_xgboost))
y_train_proba_select_xgboost = model_xgboost.predict_proba(X_train_select)[:,1]
print("AUC Score (Train): %f" % metrics.roc_auc_score(y_train_select, y_train_proba_select_xgboost))
y_proba_select_xgboost = model_xgboost.predict_proba(X_val_select)[:,1]
print("AUC Score (Test): %f" % metrics.roc_auc_score(y_val_select, y_proba_select_xgboost))
y_test_proba_xgboost = model_xgboost.predict_proba(X_test_select)[:,1]
# 建立prediction用于存放预测test数据集的结果
prediction = pd.DataFrame()
prediction['id'] = X_test_select.index
prediction['Xgboost'] = y_test_proba_xgboost

Accuracy : 0.9178
AUC Score (Train): 0.934347
AUC Score (Test): 0.880159


In [26]:
# 随机森林模型
model_RF = RandomForestClassifier(n_estimators=64,
                                  min_samples_split=100,
                                 min_samples_leaf=16, max_depth=8,
                                  max_features='sqrt',random_state=66)

model_RF.fit(X_train_select, y_train_select)
# 预测
y_pred_select_RF = model_RF.predict(X_val_select)
print("Accuracy : %.4g" % metrics.accuracy_score(y_val_select, y_pred_select_RF))
y_train_proba_select_RF = model_RF.predict_proba(X_train_select)[:,1]
print("AUC Score (Train): %f" % metrics.roc_auc_score(y_train_select, y_train_proba_select_RF))
y_proba_select_RF = model_RF.predict_proba(X_val_select)[:,1]
print("AUC Score (Test): %f" % metrics.roc_auc_score(y_val_select, y_proba_select_RF))
y_test_proba_RF = model_RF.predict_proba(X_test_select)[:,1]
prediction['RF'] = y_test_proba_RF

Accuracy : 0.9144
AUC Score (Train): 0.898657
AUC Score (Test): 0.867851


In [27]:
# 线性回归模型
model_LR = LogisticRegression(penalty='l1',
                              solver='liblinear',
                              class_weight='balanced',
                              random_state=66)
model_LR.fit(X_train_select,y_train_select)
# 预测
y_pred_select_LR = model_LR.predict(X_val_select)
print("Accuracy : %.4g" % metrics.accuracy_score(y_val_select, y_pred_select_LR))
y_train_proba_select_LR = model_LR.predict_proba(X_train_select)[:,1]
print("AUC Score (Train): %f" % metrics.roc_auc_score(y_train_select, y_train_proba_select_LR))
y_proba_select_LR = model_LR.predict_proba(X_val_select)[:,1]
print("AUC Score (Test): %f" % metrics.roc_auc_score(y_val_select, y_proba_select_LR))
y_test_proba_LR = model_LR.predict_proba(X_test_select)[:,1]
prediction['LR'] = y_test_proba_LR

Accuracy : 0.8489
AUC Score (Train): 0.852534
AUC Score (Test): 0.841010


In [ ]:
# 保存predction的数据到prediction.csv
prediction.to_csv('prediction.csv')

### 任务2
如果需要在产品中使用。你会推荐使用之前哪个算法得到的模型？解释为什么。

In [ ]:
答：由于特征的初始种类很多，且很多包含有缺失值。使用简单的逻辑回归不适合这种特征数非常多的模型。而与随机森林法相比
采用Xgboost算法能够不用对缺失值做任何处理，保持了特征的原始分布。且Xgboost能够通过特征重要度排序选择出重要的特征，
使用这些筛选的特征能够保持较高的AUC和Accuracy。因此我推荐使用Xgboost

### 任务3

任务1～任务4体现了从得到建模数据，到完成建模实验的环节。

然而，对于整个建模项目来说，以上环节只是一部分工作内容。请候选人根据自己的经验和理解，描述从接到业务部门的需求开始（假设是一个0，1分类的监督式机器学习任务），到监控模型在产品中的使用情况为止，有哪些重要的工作环节？在这些过程中，机器学习建模工程师和业务部门成员如何分工？如何合作？

建议通过流程图配合必要的文字描述，表达自己的理解。





In [31]:
答：
1.前期准备工作：不同的业务场景需要使用不同的模型，在项目开始前我们需要对业务的逻辑和需求有清晰的理解，明确好项目
周期时间和进度安排，以及模型效果的要求。这部分需要和模型需求部门深入沟通业务逻辑和时间节点。
2. 模型设计：包括模型算法的选择，单个模型还是做多模型的细分，观察期，表现期的定义，数据好坏的定义，数据的获取途径等
都要确定好。这部分主要和数据维护部门进行沟通，并和业务部门确认之前数据各阶段是否对应了业务逻辑的变化。
3. 数据拉取及清洗：根据上述观察期和表现期的定义从数据池中取数，并进行前期的数据清洗和稳定性验证工作，数据清洗包括
用户唯一性检查，缺失值检查，异常值检查等。稳定性验证主要考察变量在时间序列上的稳定性，衡量的指标有平均值/方差，
IV等。这部分工作主要与数据维护部门沟通，需要线上及线下数据库接口和相关数据库结构信息。
4. 特征工程：主要做特征的预处理和筛选，对特征进行离散化，归一化等处理，对特征进行降维，降维的方法
有IV筛选，相关性筛选，显著性筛选等。另外会基于对业务的深入理解做特征构造工作，包括特征交叉，特征转换，
对特征进行四则运算等。
5. 模型建立和评估：根据数据特点和业务需求选择合适的模型，模型建好后需要做模型评估，计算AUC,KS，并对模型做交叉验证来评估
泛化能力及模型的稳定性。
6. 模型上线部署：在业务后台上配置模型规则，对于一些复杂的模型还得需要将模型文件进行转换，并封装成一个类，用API接口等
形式来调用。这部分工作需要和业务后台部门确认线上模型的结构和各API接口。
7. 模型监控：前期主要监控模型整体及变量的稳定性，并每日观察模型规则的拒绝率与线下的差异。后期积累一定线上用户后
可评估线上模型的AUC,KS，与线下进行比较，衡量模型的线上的实际效果。


## 数据处理

### 任务4

history_df中存放了每个id，在不同time，对应的A, B的数值。

In [11]:
history_data = pd.read_csv('history_data.csv', parse_dates=['time'])


,id,time,A,B
0,1,2019-01-26 08:52:00,3.797676,8.432960
1,1,2019-01-29 14:59:00,NaN,4.157219
2,1,2019-01-13 00:01:00,6.516694,4.922487
3,1,2019-02-01 00:27:00,2.627256,NaN
4,1,2019-03-26 00:32:00,8.939391,NaN
5,2,2019-01-01 00:14:00,NaN,6.449302
6,2,2019-01-26 00:52:00,3.629338,1.529292
7,2,2019-02-04 00:05:00,4.299086,NaN
8,2,2019-02-28 00:51:00,NaN,3.140770
9,2,2019-03-01 00:44:00,6.353059,5.926424


sample_df存放了一些id和obs_time的组合。

In [12]:
sample_df = pd.read_csv('sample.csv')
sample_df

,id,obs_time
0,1,2019/3/7 0:00
1,1,2019/2/1 0:00
2,2,2019/3/15 0:00
3,2,2019/2/1 0:00


对sample_df中每个id和obs_time的组合，计算离obs_time过去10, 60天内， 该id对应的A, B的最大值, 非空记录数量。将结果存在不同的列中。

例如:


最终的结果包含的列:
* id
* obs_time
* 10天内A平均值
* 10天内A非空样本数量
* 60天内A平均值
* 60天内A非空样本数量
* 10天内B平均值
* 10天内B非空样本数量
* 60天内B平均值
* 60天内B非空样本数量

将计算结果存为excel保存

In [39]:
# 开发相应的代码
from pandas.tseries.offsets import *
history_data = pd.read_csv('history_data.csv', parse_dates=['time'])
sample_df = pd.read_csv('sample.csv', parse_dates=['obs_time'])

def process(data=history_data,sample_df=sample_df,days=10,col='A',mode='mean'):
    """
    从data数据集中筛选sample_df['obs_time']时间days日之前的所有sample_df['id']的数据，并根据mode计算col列的相应
    目标值
    :param data: dataframe 需要切片的数据集，本例为history_data
    :param df: datafram包含obs_time和id
    :param days: int,结束日期之前的天数
    :param col:要计算的列'A' or 'B'
    :param mode:计算模式 'mean' or 'max' or 'count'
    :return:list
    """
    end_time = sample_df['obs_time']
    start_time = end_time - DateOffset(days=days)
    temp = []
    for i in range(len(start_time)):
        data_1 = data[(data['time'] < end_time[i]) & (data['time'] > start_time[i])]
        if mode == 'mean':
            temp.append(data_1[data_1['id']==sample_df['id'][i]][col].fillna(0).mean())
        elif mode == 'count':
            temp.append(data_1[data_1['id'] == sample_df['id'][i]][col].count())
        elif mode == 'max':
            temp.append(data_1[data_1['id'] == sample_df['id'][i]][col].max())
        else:
            raise Exception('parameter mode error')
    return temp


days = [10, 60]
cols = ['A', 'B']
modes = ['mean', 'count']
for day in days:
    for col in cols:
        for mode in modes:
            sample_df[mode+'_of_'+col +'_in_'+ str(day)+'days'] = process(days=day,col=col,mode=mode)


# 保存数据到extract_samples
sample_df.to_csv('extract_samples.csv')
            